In [149]:
import pyodbc
import pandas as pd
import warnings

conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=./Databases/aenc.accdb;'
)
cnxn = pyodbc.connect(conn_str)
crsr = cnxn.cursor()
DBNorthWind = {'servername': 'LAPTOP-JCB37LKM\SQLEXPRESS',
      'database': 'NorthWind'}

export_conn_NorthWind = pyodbc.connect('DRIVER={SQL Server};SERVER='+ DBNorthWind['servername'] + ';DATABASE=' + DBNorthWind['database'] + ';Trusted_Connection=yes')

DBAdventureWorks = {'servername': 'LAPTOP-JCB37LKM\SQLEXPRESS',
       'database': 'AdventureWorks2019'}

export_conn_AdventureWorks = pyodbc.connect('DRIVER={SQL Server};SERVER='+ DBAdventureWorks['servername'] + ';DATABASE=' + DBAdventureWorks['database'] + ';Trusted_Connection=yes')


In [150]:
aenc_df_employee = pd.read_sql_query("SELECT emp_id,emp_fname,emp_lname,street,city,state FROM employee",cnxn)
nw_df_employee = pd.read_sql_query("SELECT EmployeeID,FirstName,LastName,Address,City,PostalCode,Country FROM employees",export_conn_NorthWind)
adv_df_employee = pd.read_sql_query("SELECT BusinessEntityID FROM HumanResources.Employee",export_conn_AdventureWorks)
adv_df_salePerson = pd.read_sql_query("SELECT BusinessEntityID FROM Sales.SalesPerson",export_conn_AdventureWorks)
adv_df_PersonPerson = pd.read_sql_query("SELECT BusinessEntityID,FirstName,LastName FROM Person.Person",export_conn_AdventureWorks)
adv_df_PersonBusinessEntity = pd.read_sql_query("SELECT BusinessEntityID FROM Person.BusinessEntity",export_conn_AdventureWorks)
adv_df_PersonBusinessEntityAdress = pd.read_sql_query("SELECT BusinessEntityID,AddressID FROM Person.BusinessEntityAddress",export_conn_AdventureWorks)
adv_df_PersonAddress = pd.read_sql_query("SELECT AddressID,AddressLine1,City,StateProvinceID,PostalCode FROM Person.Address",export_conn_AdventureWorks)
adv_df_PersonState = pd.read_sql_query("SELECT StateProvinceID,CountryRegionCode,Name FROM Person.StateProvince",export_conn_AdventureWorks)
adv_df_PersonCountryRegion = pd.read_sql_query("SELECT CountryRegionCode,Name FROM Person.CountryRegion",export_conn_AdventureWorks)




C:\Users\jboek\AppData\Local\Temp\ipykernel_2916\4102220782.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  aenc_df_employee = pd.read_sql_query("SELECT emp_id,emp_fname,emp_lname,street,city,state FROM employee",cnxn)
C:\Users\jboek\AppData\Local\Temp\ipykernel_2916\4102220782.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  nw_df_employee = pd.read_sql_query("SELECT EmployeeID,FirstName,LastName,Address,City,PostalCode,Country FROM employees",export_conn_NorthWind)
C:\Users\jboek\AppData\Local\Temp\ipykernel_2916\4102220782.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI

In [157]:
adv_salesPerson_merged = pd.merge(adv_df_salePerson,adv_df_PersonPerson,on='BusinessEntityID',how='left')
adv_salesPersonEntityAdress_merged = pd.merge(adv_salesPerson_merged,adv_df_PersonBusinessEntityAdress,on='BusinessEntityID',how='left')
adv_salesPersonAdress_merged = pd.merge(adv_salesPersonEntityAdress_merged,adv_df_PersonAddress,on='AddressID',how='left')
adv_salesPersonAdress_merged = pd.merge(adv_salesPersonAdress_merged,adv_df_PersonState,on="StateProvinceID",how="left")
adv_salesPersonAdress_merged = pd.merge(adv_salesPersonAdress_merged,adv_df_PersonCountryRegion,on="CountryRegionCode",how="left")
adv_salesPersonAdress_merged = adv_salesPersonAdress_merged.rename(columns={
    'BusinessEntityID' : 'EmployeeID',
    'AddressLine1' : 'Address',  
    'Name_x' : 'State',  
    'Name_y' : 'Country',  
})
adv_salesPersonAdress_merged = adv_salesPersonAdress_merged.drop(['AddressID','PostalCode','CountryRegionCode','StateProvinceID'],axis=1)
adv_salesPersonAdress_merged['Source'] = 'AdventureWorks'
adv_df_final = adv_salesPersonAdress_merged
adv_df_final


,EmployeeID,FirstName,LastName,Address,City,State,Country,Source
0,285,Syed,Abbas,7484 Roundtree Drive,Bothell,Washington,United States,AdventureWorks
1,283,David,Campbell,2284 Azalea Avenue,Bellevue,Washington,United States,AdventureWorks
2,278,Garrett,Vargas,10203 Acorn Avenue,Calgary,Alberta,Canada,AdventureWorks
3,279,Tsvi,Reiter,8291 Crossbow Way,Memphis,Tennessee,United States,AdventureWorks
4,277,Jillian,Carson,80 Sunview Terrace,Duluth,Minnesota,United States,AdventureWorks
5,275,Michael,Blythe,8154 Via Mexico,Detroit,Michigan,United States,AdventureWorks
6,288,Rachel,Valdez,Pascalstr 951,Berlin,Hamburg,Germany,AdventureWorks
7,287,Amy,Alberts,5009 Orange Street,Renton,Washington,United States,AdventureWorks
8,276,Linda,Mitchell,2487 Riverside Drive,Nevada,Utah,United States,AdventureWorks
9,282,José,Saraiva,9100 Sheppard Avenue North,Ottawa,Ontario,Canada,AdventureWorks


In [152]:
nw_df_employee['Country'] = nw_df_employee['Country'].replace(['USA', 'UK'], ['US', 'GB'])
nw_df_employee = nw_df_employee.rename(columns={
     'Country': 'CountryRegionCode',
 })
nw_df_employee = pd.merge(nw_df_employee,adv_df_PersonCountryRegion,on='CountryRegionCode',how='left')


nw_df_employee = nw_df_employee.drop(['CountryRegionCode'],axis=1)
nw_df_employee = nw_df_employee.rename(columns={
    'Name': 'Country'
})


In [153]:
merged_df = pd.merge(adv_df_PersonState, adv_df_PersonAddress, on='StateProvinceID', how='inner')
merged_df = pd.merge(merged_df, adv_df_PersonCountryRegion, on='CountryRegionCode', how='left')
merged_df = merged_df.drop(columns=['StateProvinceID','CountryRegionCode','AddressID','AddressLine1','PostalCode'])
merged_df = merged_df.rename(columns={'Name_y' : 'Country','Name_x': 'State'})
merged_df = merged_df.drop_duplicates()
merged_df

,State,City,Country
0,Alberta,Calgary,Canada
7,Alberta,Edmonton,Canada
25,Alabama,Huntsville,United States
26,Alabama,Birmingham,United States
27,Alabama,Mobile,United States
...,...,...,...
19269,Seine Saint Denis,Pantin,France
19271,Seine Saint Denis,Bobigny,France
19276,Seine Saint Denis,Drancy,France
19552,Val de Marne,Orly,France


In [154]:
df_nw_final = pd.merge(nw_df_employee,merged_df,left_on=['City','Country'],right_on=['City','Country'],how='left')
df_nw_final = df_nw_final.drop(columns=['PostalCode'])
df_nw_final = df_nw_final.reindex(columns=['EmployeeID','FirstName','LastName','Address','City','State','Country'])
df_nw_final['Source'] = 'NorthWind'
df_nw_final

,EmployeeID,FirstName,LastName,Address,City,State,Country,Source
0,1,Nancy,Davolio,507 - 20th Ave. E.Apt. 2A,Seattle,Washington,United States,NorthWind
1,2,Andrew,Fuller,908 W. Capital Way,Tacoma,Washington,United States,NorthWind
2,3,Janet,Leverling,722 Moss Bay Blvd.,Kirkland,Washington,United States,NorthWind
3,4,Margaret,Peacock,4110 Old Redmond Rd.,Redmond,Washington,United States,NorthWind
4,5,Steven,Buchanan,14 Garrett Hill,London,England,United Kingdom,NorthWind
5,6,Michael,Suyama,Coventry House\nMiner Rd.,London,England,United Kingdom,NorthWind
6,7,Robert,King,Edgeham Hollow\nWinchester Way,London,England,United Kingdom,NorthWind
7,8,Laura,Callahan,4726 - 11th Ave. N.E.,Seattle,Washington,United States,NorthWind
8,9,Anne,Dodsworth,7 Houndstooth Rd.,London,England,United Kingdom,NorthWind


In [158]:
aenc_df_employee_renamed = aenc_df_employee.rename(columns={'emp_id': 'EmployeeID','emp_fname':'FirstName','emp_lname':'LastName','street':'Address','city':'City','state':'State'})
aenc_df_employee_renamed['Country'] = 'United States'

state_mapping = {"MA": "Massachusetts", "GA": "Georgia", "CA": "California", "TX": "Texas"}
aenc_df_employee_renamed['State'] = aenc_df_employee_renamed['State'].replace(state_mapping)
aenc_df_employee_renamed['Source'] = 'AenC'
fd_aenc_final = aenc_df_employee_renamed
fd_aenc_final



,EmployeeID,FirstName,LastName,Address,City,State,Country,Source
0,102,Fran,Whitney,49 East Washington Street,Needham,Massachusetts,United States,AenC
1,105,Matthew,Cobb,77 Pleasant Street,Waltham,Massachusetts,United States,AenC
2,129,Philip,Chin,59 Pond Street,Atlanta,Georgia,United States,AenC
3,148,Julie,Jordan,144 Great Plain Avenue,Winchester,Massachusetts,United States,AenC
4,160,Robert,Breault,58 Cherry Street,Milton,Massachusetts,United States,AenC
...,...,...,...,...,...,...,...,...
70,1643,Elizabeth,Lambert,29 Union Street,Burlington,Massachusetts,United States,AenC
71,1658,Michael,Lynch,76 Brookside Road,Waltham,Massachusetts,United States,AenC
72,1684,Janet,Hildebrand,47 Hilltop Street,Waltham,Massachusetts,United States,AenC
73,1740,Robert,Nielsen,55 Sargent Avenue,Bedford,Massachusetts,United States,AenC


In [159]:
combined_employee = pd.concat([adv_df_final, fd_aenc_final, df_nw_final]).reset_index(drop=True)
combined_employee['EmployeeKey'] = combined_employee.index + 1
combined_employee = combined_employee.reindex(columns=['EmployeeKey','EmployeeID', 'FirstName','LastName','Address','City','State','Country','Source'])
combined_employee

,EmployeeKey,EmployeeID,FirstName,LastName,Address,City,State,Country,Source
0,1,285,Syed,Abbas,7484 Roundtree Drive,Bothell,Washington,United States,AdventureWorks
1,2,283,David,Campbell,2284 Azalea Avenue,Bellevue,Washington,United States,AdventureWorks
2,3,278,Garrett,Vargas,10203 Acorn Avenue,Calgary,Alberta,Canada,AdventureWorks
3,4,279,Tsvi,Reiter,8291 Crossbow Way,Memphis,Tennessee,United States,AdventureWorks
4,5,277,Jillian,Carson,80 Sunview Terrace,Duluth,Minnesota,United States,AdventureWorks
...,...,...,...,...,...,...,...,...,...
96,97,5,Steven,Buchanan,14 Garrett Hill,London,England,United Kingdom,NorthWind
97,98,6,Michael,Suyama,Coventry House\nMiner Rd.,London,England,United Kingdom,NorthWind
98,99,7,Robert,King,Edgeham Hollow\nWinchester Way,London,England,United Kingdom,NorthWind
99,100,8,Laura,Callahan,4726 - 11th Ave. N.E.,Seattle,Washington,United States,NorthWind
